In [ ]:
import re
from collections import namedtuple
from random import randrange

In [ ]:
with open("../data/2025/day12.txt") as f:
    data = f.read()

In [ ]:
data = """0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2"""

In [ ]:
Shape = namedtuple('Shape', 'w h tiles')
Region = namedtuple('Region', 'w h items')

def shape_parse(shape_schema: list) -> Shape:
    w, h, shape_dict = len(shape_schema[0]), len(shape_schema), {}
    for tile_y, row in enumerate(shape_schema):
        for tile_x, tile in enumerate(row):
            if tile != '.': shape_dict[tile_x + tile_y * 1j] = tile
    return Shape(w, h, shape_dict)

def shape_rotate_90(shape: Shape, turns: int) -> Shape:
    center = shape.w // 2 + shape.h // 2 * 1j
    r = 1j ** turns
    return Shape(shape.w, shape.h, {(xy-center) * r + center: tile for xy, tile in shape.tiles.items()})

def shape_translate(shape: Shape, offset: complex) -> Shape:
    return Shape(
        shape.w,
        shape.h,
        {(xy + offset): tile for xy, tile in shape.tiles.items()}
    )

def shape_flip(shape: Shape, axis: str="x"):
    center = shape.w // 2 + shape.h // 2 * 1j
    return Shape(
        shape.w,
        shape.h,
        {(-1 if axis=='y' else 1) * (xy-center).conjugate() + center: tile for xy, tile in shape.tiles.items()}
    )

# [TODO] Generator (yield)
# [TODO] We can do this once since it's not relative

def shape_transforms(shape: Shape):
    results = []
    for flip_shape in [shape, shape_flip(shape, 'x'), shape_flip(shape, 'y')]:
       for turn in range(4):
            new_shape = shape_rotate_90(flip_shape, turn)
            if new_shape not in results: results.append(new_shape)
    return results

In [ ]:
sections = data.split("\n\n")
shape_schemas = [line.splitlines()[1:] for line in sections[:-1]]
regions = [Region(*(int(w), int(h), [int(i) for i in items.split(' ')]))
           for line in sections[-1].splitlines()
           for w,h,items in re.findall(r'(\d+)x(\d+): (.*)', line)]
shapes = [shape_parse(shape) for shape_id, shape in enumerate(shape_schemas)]
shape_transforms_cache = [shape_transforms(shape) for shape_id, shape in enumerate(shapes)]

In [ ]:
# [TODO] Hybrid shapes
#shapes.append(Shape(
#    w=4, h=4, tiles={}
#))

In [ ]:
# def get_next_open_coord(box: set):
#     for xy in [x+y*1j for y in range(region.h) for x in range(region.w)]:
#         if xy not in box: return xy
#     return None

# neighbor_bounds = frozenset([x+y*1j for y in range(-1,4) for x in range(-1,4)])
# def get_neighbors_for_shape(shape: Shape, box: set) -> int:
#     coords = neighbor_bounds.difference(set(shape.tiles.keys()))
#     return len(coords)

def fits_in_box(shape_to_fit: Shape, box: set) -> tuple[bool, frozenset, int]:
    # [TODO] Store only the frozenset in the first place (no tiles)
    shape_coords = frozenset(shape_to_fit.tiles.keys())
    shape_in_bounds = box_bounds.issuperset(shape_coords)
    shape_not_overlapping = box.isdisjoint(shape_coords)

    # [TODO] Score our combination
    # score by +1 for external neighbors on edges?
    score = -sum(box | shape_coords).imag

    # score = 0

    if shape_in_bounds and shape_not_overlapping:
        return True, shape_coords, score

    return False, frozenset(), 0

# def has_3x3_slot(box):
#     cells = [x+y*1j for y in range(region.h-2) for x in range(region.w-2)]
#     return False

def pack_box(box: set, shapes_to_fit: dict[int,int], packed_shapes: list[Shape]):
    if 0 == sum(shapes_to_fit.values()):
        #print("Done!", packed_shapes)
        return box # return packed_shapes

    # [TODO] This is expensive to do every recursion step
    search_coords = [x+y*1j for y in range(region.h-2) for x in range(region.w-2)]

    for next_open_xy in search_coords: # [TODO] next coordinate
        # [TODO] If no shapes would fit, abort
        # [TODO] This should ignore non-adjacent spaces
        # [TODO] Only test distinct shapes with our counters

        if next_open_xy in box: continue

        # [TODO] If we don't have a 3x3 space available abort

        # print("Trying", next_open_xy)
        # for grid_y in range(region.h):
        #     for grid_x in range(region.w):
        #         print('#' if (grid_x + grid_y * 1j) in box else '.', end='')
        #     print('')
        # print('')

        #print("Finding shape at", next_open_xy, "with", shapes_to_fit.items(), "remaining, depth", len(packed_shapes))
        #print("Used coords in box", box)
        #print("Available coords", next_open_xy, search_coords)

        candidates = []

        for shape_id, shape_quantity in shapes_to_fit.items():
            #print("Gathering Shape ID", shape_id, "Quantity", shape_quantity)
            #if shape_quantity == 0: continue
            #shape_to_fit = shapes[shape_id]
            #if region.w * region.h - len(box) < len(shape_to_fit.tiles): continue

            for shape_state in shape_transforms_cache[shape_id]:
                shape_state = shape_translate(shape_state, next_open_xy)
                # [TODO] Score all shape states (minimize voids, etc)

                success, coords, score = fits_in_box(shape_state, box)
                if success:
                    #print("Adding candidate")
                    candidates.append((score, shape_id, shape_quantity, coords, shape_state))

                #if (result := fits_in_box(shape_state, box))[0]:
                #    next_shapes_to_fit = shapes_to_fit.copy().update({shape_id: shape_quantity - 1})
                #    if packed := pack_box(box | result[1], next_shapes_to_fit, packed_shapes + [shape_state]):
                #        return packed

        #print("Candidates", candidates)
        if not candidates: continue

        for score, shape_id, shape_quantity, candidate, shape_state in sorted(candidates, reverse=True):
        #for score, shape_id, shape_quantity, candidate, shape_state in candidates:
            if shape_quantity == 0: continue

            #print("Placing next candidate", shape_id, shape_quantity)
            next_shapes_to_fit = shapes_to_fit.copy()
            next_shapes_to_fit[shape_id] -= 1
            #print(next_shapes_to_fit)

            if packed := pack_box(box | candidate, next_shapes_to_fit, packed_shapes + [shape_state]):
                return packed

            #print("Trying the next candidate...")

            #next_shapes_to_fit[shape_id] += 1
            # If we hit this we backtracked

        #print("Reached an impossible state.")
        #return None # We couldn't fit this shape anywhere after backtracking

        # [TODO] If we're moving to the next coordinate, ensure we have room to place

    #print("Searched entire grid without finding a solution.")
    return None

In [ ]:
for region in regions[2:3]:
    shapes_to_pack = {item: quantity for item, quantity in enumerate(region.items) if quantity > 0}

    print(region)

    shape_sort_order = [4,1,5,0,2,3]
    shapes_to_pack = dict(sorted(shapes_to_pack.items(), key=lambda x: shape_sort_order.index(x[0])))
    print("Shapes to pack", shapes_to_pack)

    box = set()
    box_bounds = frozenset([x+y*1j for y in range(region.h) for x in range(region.w)])
    packed_box = pack_box(box, shapes_to_pack, [])

    if packed_box:
        for grid_y in range(region.h):
            for grid_x in range(region.w):
                print('#' if grid_x + grid_y * 1j in packed_box else '.', end='')
            print('')
        print('')
        print("Packed!", packed_box)
    else:
        print("Unable to pack!")

In [ ]:
#from random import randrange

# w=3 h=7 items=0,0,0,2,0,1
# w=7 h=3 items=0,0,0,2,0,1
# w=3 h=14 items=0,0,0,4,0,2
# w=14 h=3 items=0,0,0,4,0,2
# w=6 h=7 items=0,0,0,4,0,2
# w=7 h=6 items=0,0,0,4,0,2
# w=3 h=7 items=1,1,0,1,0,0
# w=7 h=3 items=1,1,0,1,0,0

#while True:
for _ in range(1):
    #region = Region(w=4, h=4, items=[randrange(0, 2) for _ in range(6)])
    #region = Region(w=7, h=6, items=[0,0,0,4,0,2])
    region = Region(w=10, h=10, items=[0,0,0,0,2,0])

    #47x39: 64 44 38 43 38 53
    #region = Region(w=47, h=39, items=[64,44,38,43,38,53])

    box = set()
    box_bounds = frozenset([x+y*1j for y in range(region.h) for x in range(region.w)])

    #shapes_to_pack = [x for sublist in [[shape_id]*quantity
    #    for shape_id, quantity in enumerate(region.items) if quantity > 0]
    #    for x in sublist]
    #shuffle(shapes_to_pack)

    shapes_to_pack = {item: quantity for item, quantity in enumerate(region.items) if quantity > 0}

    #shape_sort_order = [4,1,5,0,2,3]
    #shapes_to_pack = sorted(shapes_to_pack, key=lambda x: shape_sort_order.index(x))
    #print(shapes_to_pack)

    packed_box = pack_box(box, shapes_to_pack, [])

    if packed_box:
        for grid_y in range(region.h):
            for grid_x in range(region.w):
                print('#' if grid_x + grid_y * 1j in packed_box else '.', end='')
            print('')
        print('')
        print("Packed!", packed_box)
        print(region)
        if len(packed_box) == region.w * region.h:
            print("PERFECT!"); break
    else:
        pass #print("Unable to pack!")

In [ ]:
from random import randrange


In [ ]:
regions[2]

In [ ]:
#print(get_neighbors_for_shape(shapes[0]))
#print(len(shapes[0].tiles.keys()))
#print(len(neighbor_bounds))
#print(len(neighbor_bounds.difference(set(shapes[0].tiles.keys()))))

In [ ]:
print(box)

In [ ]:
frozenset([x+y*1j for y in range(-1,4) for x in range(-1,4)])